In [ ]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [ ]:
df = pd.read_csv("out_without_nan_mood_normalised.csv")
df['mood_mean_TARGET'] = df['mood_mean_TARGET'].astype(float)
df = pd.concat([df,pd.get_dummies(df['id'])],axis=1)
df = pd.concat([df,pd.get_dummies(df['weekDay'],prefix="day_")],axis=1)
df = df.drop(['weekDay_time_5','day_time_5','weekDay_time_4','day_time_4','weekDay_time_3','day_time_3','weekDay_time_2','day_time_2','weekDay_time_1','day_time_1','weekDay_time_5','circumplex.valence_mean_time_1'],axis=1)

Lets remove highly correlated features

In [ ]:
def correlation(dataset, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset
    
    return(dataset)
newDF =df.drop(["mood_mean_TARGET"],axis=1)
correlation(newDF,0.5)
newDF['mood_mean_TARGET'] = df['mood_mean_TARGET']
df = newDF

In [ ]:
corr=df.corr()
goodColumns= corr[abs(corr['mood_mean_TARGET'])>0.2]['mood_mean_TARGET'].sort_values()
goodColumns = goodColumns.drop(['mood_mean_TARGET'])

usedColumns = goodColumns.keys()

#usedColumns = ['AS14.01', 'AS14.02', 'AS14.03', 'AS14.05',
#       'AS14.06', 'AS14.07', 'AS14.08', 'AS14.09', 'AS14.12', 'AS14.13',
#       'AS14.14', 'AS14.15', 'AS14.16', 'AS14.17', 'AS14.19', 'AS14.20',
#       'AS14.23', 'AS14.24', 'AS14.25', 'AS14.26', 'AS14.27', 'AS14.28',
#       'AS14.29', 'AS14.30', 'AS14.31', 'AS14.32', 'AS14.33', 'day__0.0',
#       'day__0.16666666666666666', 'day__0.3333333333333333', 'day__0.5',
#       'day__0.6666666666666666', 'day__0.8333333333333334', 'day__1.0','mood_mean','mood_mean_time_2','mood_mean_time_3','circumplex.arousal_mean','circumplex.arousal_mean_time_2','circumplex.arousal_mean_time_3','circumplex.valence_mean','circumplex.valence_mean_time_2','circumplex.valence_mean_time_3']
#usedColumns=['mood_mean','mood_mean_time_2','mood_mean_time_3','circumplex.arousal_mean','circumplex.arousal_mean_time_2','circumplex.arousal_mean_time_3','circumplex.valence_mean','circumplex.valence_mean_time_2','circumplex.valence_mean_time_3']

In [ ]:
usedColumns

Index(['AS14.07', 'lastTimeSeenMood_7', 'AS14.30',
       'numberOfTimesSeenMood5Days_8', 'circumplex.valence_mean',
       'mood_mean_time_3'],
      dtype='object')

In [ ]:
notRelevantAll=df.drop(['id','date'],axis=1).dropna()
goldY= notRelevantAll['mood_mean_TARGET']
relevant = notRelevantAll.drop(['mood_mean_TARGET'],axis=1)


In [ ]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(len(usedColumns), input_dim=len(usedColumns), kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=1)

In [ ]:
scoring = {'R2': 'r2',
           'Mean squared Error': 'neg_mean_squared_error',
           'Explained variance': 'explained_variance'}

scores = cross_validate(estimator, relevant[usedColumns], goldY, cv=7,scoring=scoring)
print("R2: %0.2f (+/- %0.2f)" % (scores['test_R2'].mean(), scores['test_R2'].std() * 2))
print("MSE: %0.2f (+/- %0.2f)" % (scores['test_Mean squared Error'].mean(), scores['test_Mean squared Error'].std() * 2))
print("EXPLAINED VARIANCE: %0.2f (+/- %0.2f)" % (scores['test_Explained variance'].mean(), scores['test_Explained variance'].std() * 2))


Epoch 1/100
1042/1042 [==============================] - 0s 240us/step - loss: 41.8443
Epoch 2/100
1042/1042 [==============================] - 0s 103us/step - loss: 17.2313
Epoch 3/100
1042/1042 [==============================] - 0s 91us/step - loss: 9.1019
Epoch 4/100
1042/1042 [==============================] - 0s 91us/step - loss: 5.6204
Epoch 5/100
1042/1042 [==============================] - 0s 93us/step - loss: 3.1593
Epoch 6/100
1042/1042 [==============================] - 0s 92us/step - loss: 1.6297
Epoch 7/100
1042/1042 [==============================] - 0s 94us/step - loss: 0.8786
Epoch 8/100
1042/1042 [==============================] - 0s 94us/step - loss: 0.6083
Epoch 9/100
1042/1042 [==============================] - 0s 96us/step - loss: 0.5327
Epoch 10/100
1042/1042 [==============================] - 0s 92us/step - loss: 0.5164
Epoch 11/100
1042/1042 [==============================] - 0s 93us/step - loss: 0.5114
Epoch 12/100
1042/1042 [==============================] - 0